<a href="https://colab.research.google.com/github/brendenwest/cis276/blob/main/7_data_analysis_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Wrangling

### Reading
- Murach's, Chapter 7, 8
- https://wesmckinney.com/book/data-wrangling.html

### Tutorials
- https://www.datacamp.com/community/tutorials/pandas-multi-index
- https://www.datacamp.com/community/tutorials/pandas-split-apply-combine-groupby

### Learning Outcomes

- How to create and plot long data
- how to combine & reshape DataFrames  
- How to group & aggregate data
- How to create & use pivot tables
- How to work with bins


## Creating Long Data

Sometimes it's useful to analyze data in `long` format, where data from multiple source columns are combined into just two columns - a feature identifier and a feature value.

The Pandas `melt()` method combines specified columns into two new columns, where each row contains the original column name and the corresponding value.


In [ ]:
import pandas as pd
# get example data
download_url = ("https://raw.githubusercontent.com/fivethirtyeight/data/master/college-majors/recent-grads.csv")
salaries = pd.read_csv(download_url)
salaries[salaries['Major_category'] == 'Engineering'].sort_values(by=['Major_code']).head(10)

,Rank,Major_code,Major,Total,Men,Women,Major_category,ShareWomen,Sample_size,Employed,...,Part_time,Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
58,59,1401,ARCHITECTURE,46420.0,25463.0,20957.0,Engineering,0.451465,362,34158,...,10206,20026,4366,0.113332,40000,31000,50000,16178,13724,4221
17,18,2400,GENERAL ENGINEERING,61152.0,45683.0,15469.0,Engineering,0.252960,425,44931,...,7199,33540,2859,0.059824,56000,36000,69000,26898,11734,3192
11,12,2401,AEROSPACE ENGINEERING,15058.0,12953.0,2105.0,Engineering,0.139793,147,11391,...,2724,8790,794,0.065162,60000,42000,70000,8184,2425,372
15,16,2402,BIOLOGICAL ENGINEERING,8925.0,6062.0,2863.0,Engineering,0.320784,55,6170,...,1983,3413,589,0.087143,57100,40000,76000,3603,1595,524
18,19,2403,ARCHITECTURAL ENGINEERING,2825.0,1835.0,990.0,Engineering,0.350442,26,2575,...,343,1848,170,0.061931,54000,38000,65000,1665,649,137
12,13,2404,BIOMEDICAL ENGINEERING,14955.0,8407.0,6548.0,Engineering,0.437847,79,10047,...,2694,5986,1019,0.092084,60000,36000,70000,6439,2471,789
4,5,2405,CHEMICAL ENGINEERING,32260.0,21239.0,11021.0,Engineering,0.341631,289,25694,...,5180,16697,1672,0.061098,65000,50000,75000,18314,4440,972
25,26,2406,CIVIL ENGINEERING,53153.0,41081.0,12072.0,Engineering,0.227118,565,43041,...,10080,29196,3270,0.070610,50000,40000,60000,28526,9356,2899
10,11,2407,COMPUTER ENGINEERING,41542.0,33258.0,8284.0,Engineering,0.199413,399,32506,...,5146,23621,2275,0.065409,60000,45000,75000,23694,5721,980
9,10,2408,ELECTRICAL ENGINEERING,81527.0,65511.0,16016.0,Engineering,0.196450,631,61928,...,12695,41413,3895,0.059174,60000,45000,72000,45829,10874,3170


In [ ]:

salaries_melted = pd.melt(salaries, id_vars=['Major_code', 'Major_category','Unemployment_rate'],
value_vars=['Men', 'Women'],
var_name='Gender', value_name='count')

df = salaries_melted[salaries_melted['Major_category'] == 'Engineering']
df.sort_values(by=['Major_code']).head(10)

,Major_code,Major_category,Unemployment_rate,Gender,count
58,1401,Engineering,0.113332,Men,25463.0
231,1401,Engineering,0.113332,Women,20957.0
190,2400,Engineering,0.059824,Women,15469.0
17,2400,Engineering,0.059824,Men,45683.0
184,2401,Engineering,0.065162,Women,2105.0
11,2401,Engineering,0.065162,Men,12953.0
188,2402,Engineering,0.087143,Women,2863.0
15,2402,Engineering,0.087143,Men,6062.0
18,2403,Engineering,0.061931,Men,1835.0
191,2403,Engineering,0.061931,Women,990.0


## Combining & Merging Datasets

pandas supports several different ways to combine datasets:

- `pandas.merge` - connects rows in DataFrames based on one or more keys, like a SQL database join operation
- `pandas.concat` - concatenates or 'stacks' data along an axis
- `combine_first` instance method enables splicing together overlapping data to fill in missing values in one object with values from another.


### Combining data with overlap

`combine_first` is similar to NumPy's `where` method for performing ternary operations.


### Database-style joins

Merge or join operations combine datasets by linking rows using one or more keys.

If column to join on is not specified, `merge` uses the overlapping column names as the keys.

Different column names from each dataset can be specified as keys.

By default `merge` does an `inner` join - the keys in the result are the intersection, or the common set found in both tables. Other possible options are `left`, `right`, and `outer` joins.

To merge with multiple keys, pass a list of column names.

`left_index=True` or `right_index=True` (or both) can indicate that the index should be used as the merge key

DataFrame has a convenient `join` instance method for merging by index. It can also be used to combine together many DataFrame objects having the same or similar indexes but non-overlapping columns.

### Concatenation

`concat` provides a consistent way to:
- combine objects that are indexed differently
- make the combined data identifiable in the resulting objects
- preserve data in the concatenation axis

By default `concat` works along axis=0, producing another Series. If you pass axis=1, the result will instead be a DataFrame (axis=1 is the columns)

Concatenation along axis=1 supports an argument for type of `join` to use.

Concatenation can create a hierarchical index on the concatenation axis using the `keys` argument.

`ignore_index=True` allows drops the original indexes from the result.


## General split-apply-combine

#### Suppressing Group Keys
#### Quantile & Bucket Analysis
#### Filling missing values
#### Random sampling & permutation
#### Group weighted average and correlation

## Grouping Data

The expressiveness of Python and pandas allows complex group operations using any function that accepts a pandas object or NumPy array. This can include:

- Splitting a pandas object into pieces using one or more keys
- Calculating group summary statistics
- Applying within-group transformations or other manipulations
- Computing pivot tables and cross-tabulations
- Performing quantile analysis and other statistical group analyses

### GroupBy Operations

Group operations involve the `split-apply-combine` mechanism.

1. Data are split into groups based on one or more keys
2. A function is applied to each group
3. Results of the function application are combined into a new object

Grouping keys can take many forms, and the keys do not have to be all of the same type.

pandas `groupby` method returns a GroupBy object that can be re-used.

DataFrame columns can be used as the group keys.

Numeric aggregations will exclude `nuisance` (non-numeric) columns from the result

By default `groupby` groups on axis=0, but can group on any of the other axes.

### Iterating over groups

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data.

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation. This means that:
```
df.groupby('key1')['data1']
```
is essentially the same as:
```
df['data1'].groupby(df['key1'])
```

#### Grouping with Series or Dicts


  #### Grouping with Functions
  #### Grouping by Index Levels

## Pivot Tables & Cross-Tabulation

A pivot table is a data summarization tool that aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns.

### Pivot Tables
DataFrame has a `pivot_table` method, and there is also a top-level `pandas.pivot_table` function.

The `margins=True` argument computes partial totals across rows and columns of the result table.

### Cross-Tabulations

A cross-tabulation (or crosstab) is a special case of a pivot table that computes group frequencies.